[E8]Movielens 영화 추천 실습

1. 분석
 - ratings에 있는 유니크한 영화 개수
 - rating에 있는 유니크한 사용자 수
 - 가장 인기 있는 영화 30개(인기순)
2. 내가 선호하는 영화 5가지 골라서 rating 추가
3. CSR Matrix 직접 만들어보기
4. als_model = AlternatingLeastSquares 모델을 직접 구성 및 훈련
5. 내가 선호하는 영화 중 하나, 그 외 영화 하나 골라 훈련된 모델이 예측한 나의 선호파악
6. 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다
7. 내가 가장 좋아할만한 영화들을 추천받아 봅시다.
 

In [1]:
import pandas as pd
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

In [4]:
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [5]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
#movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding = "ISO-8859-1")

movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


- ratings에 있는 유니크한 영화 개수
- rating에 있는 유니크한 사용자 수
- 가장 인기 있는 영화 30개(인기순)

In [6]:
# ratings에 있는 유니크한 영화 개수
ratings['movie_id'].nunique()

3628

In [7]:
# ratings에 있는 유니크한 사용자 수
ratings['user_id'].nunique()

6039

In [8]:
movies_data = pd.merge(left = ratings , right = movies, how = "inner", on = "movie_id")
movies_data.head(10)


,user_id,movie_id,count,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,One Flew Over the Cuckoo's Nest (1975),Drama
5,18,1193,4,978156168,One Flew Over the Cuckoo's Nest (1975),Drama
6,19,1193,5,982730936,One Flew Over the Cuckoo's Nest (1975),Drama
7,24,1193,5,978136709,One Flew Over the Cuckoo's Nest (1975),Drama
8,28,1193,3,978125194,One Flew Over the Cuckoo's Nest (1975),Drama
9,33,1193,5,978557765,One Flew Over the Cuckoo's Nest (1975),Drama


In [9]:
movies_data.isnull().sum()

user_id      0
movie_id     0
count        0
timestamp    0
title        0
genre        0
dtype: int64

In [10]:
# 평점 3,4,5 모두 인기있는 경우라고 정한다.
# 3,4,5점을 한번 받으면 1 count로 정하고, 각 영화에 대한 총 count수를 로 정렬한다. 
movies_count = movies_data.groupby('title')['count'].count()

movies_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

내가 선호하는 영화를 5가지 골라서 rating에 추가해 줍시다.

In [11]:
# [title] input -> [movie_id] output
def title_in_id_out(title):
    a=movies_data[movies_data['title'].isin([title])] # title칼럼중 인자로 들어온 title과 같은 행렬 추출
    mv_id=a.iloc[0,1] #추출된 행렬중 0번째 행의 1번째칼럼[movie_id] 값 가져온다.
    return mv_id
#title가 'Jumanji (1995)'인 movie_id 출력
title_in_id_out('Jumanji (1995)')

2

In [12]:
# [movie_id] output -> [title] input
def id_in_title_out(movie_id):
    a=movies_data[movies_data['movie_id'].isin([movie_id])] # movie_id칼럼중 인자에 들어온 영화id와 같은 행렬 추출
    title=a.iloc[0,4] #추출된 행렬중 0번째 행의 4번째칼럼[title] 값 가져온다.
    return title

#moive_id가 23인 title값 출력
id_in_title_out(23)

'Assassins (1995)'

In [16]:
# 영화 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰주세요. 
#선호 영화 리스트 생성
my_favorite = ['Jumanji (1995)', 'Ace Ventura: When Nature Calls (1995)', 'Apollo 13 (1995)', 'Home Alone 2: Lost in New York (1992)' ,'Top Gun (1986)']
my_movie_id = []


#선호 영화 딕셔너리 생성
dir_favorite ={} #{'title' :'movie_id'} 딕셔너리
 
for title in my_favorite:
    mv_id = title_in_id_out(title)
    my_movie_id.append(mv_id) 
    print('영화명:',title,'\t\t 무비id:', mv_id)
    
    dir_favorite[title]=int(mv_id)
    print('영화명:',title,'\t\t 무비id:', dir_favorite[title])
    
       

#print(movies_data.columns)
#movies_data.sort_values(by='movie_id')
my_movie_id

영화명: Jumanji (1995) 		 무비id: 2
영화명: Jumanji (1995) 		 무비id: 2
영화명: Ace Ventura: When Nature Calls (1995) 		 무비id: 19
영화명: Ace Ventura: When Nature Calls (1995) 		 무비id: 19
영화명: Apollo 13 (1995) 		 무비id: 150
영화명: Apollo 13 (1995) 		 무비id: 150
영화명: Home Alone 2: Lost in New York (1992) 		 무비id: 2953
영화명: Home Alone 2: Lost in New York (1992) 		 무비id: 2953
영화명: Top Gun (1986) 		 무비id: 1101
영화명: Top Gun (1986) 		 무비id: 1101


[2, 19, 150, 2953, 1101]

In [17]:
# user_id 마지막은 6040
# user_id은 1부터 시작해서 6040개로 끝난다면 유니크한 값은 6040개가 되어야되는데 6039개다 ,그럼 중간에 한개의 값이 없다고 추측된다.
print(ratings['user_id'].nunique()) 
movies_data.sort_values(by='user_id',ascending=True, inplace=False)
6039

6039


6039

In [19]:
#my_moivelist = pd.DataFrame({'user_id': [0]*5, 'movie_id': my_movie_id, 'count':[5]*5})
my_moivelist = pd.DataFrame({'user_id': ['JJEONDA']*5, 'movie_id': my_movie_id, 'count':[5]*5})

# 사용자는 6039이였기때문에  나에대한 유저번호는 6040으로 설정한다.
if not ratings.isin({'user_id':[6041]})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    ratings = ratings.append(my_moivelist)              

ratings.tail(10)

,user_id,movie_id,count,timestamp
0,syj,2,5,NaN
1,syj,19,5,NaN
2,syj,150,5,NaN
3,syj,2953,5,NaN
4,syj,1101,5,NaN
0,JJEONDA,2,5,NaN
1,JJEONDA,19,5,NaN
2,JJEONDA,150,5,NaN
3,JJEONDA,2953,5,NaN
4,JJEONDA,1101,5,NaN


In [20]:

print('기본행 개수',len(movies_data))
movies_data = pd.merge(left = ratings , right = movies, how = "inner", on = "movie_id")
movies_data.head(10)
print('추가후 행 개수',len(movies_data))
movies_data.tail(5)

기본행 개수 836478
추가후 행 개수 836488


,user_id,movie_id,count,timestamp,title,genre
836483,5851,3607,5,9.577566e+08,One Little Indian (1973),Comedy|Drama|Western
836484,5854,3026,4,9.583469e+08,Slaughterhouse (1987),Horror
836485,5854,690,3,9.577443e+08,"Promise, The (Versprechen, Das) (1994)",Romance
836486,5938,2909,4,9.572734e+08,"Five Wives, Three Secretaries and Me (1998)",Documentary
836487,5948,1360,5,1.016564e+09,Identification of a Woman (Identificazione di ...,Drama


In [21]:
movies_tmp= movies_data.loc[:,['user_id','movie_id','count']]
movies_tmp

,user_id,movie_id,count
0,1,1193,5
1,2,1193,5
2,12,1193,4
3,15,1193,4
4,17,1193,5
...,...,...,...
836483,5851,3607,5
836484,5854,3026,4
836485,5854,690,3
836486,5938,2909,4


In [22]:
# 고유한 유저, movie_id를 찾아내는 코드
user_unique = movies_tmp['user_id'].unique()
movie_unique = movies_tmp['movie_id'].unique()

# 유저id, 영화id indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [23]:
num=0
for key, value in user_to_idx.items():
    print('user_id:{}    ,user_idx:{}'.format(key, value))
    num+=1
    if num>5:
        break
print('user_to_idx의 사이즈:', len(user_to_idx))

user_id:1    ,user_idx:0
user_id:2    ,user_idx:1
user_id:12    ,user_idx:2
user_id:15    ,user_idx:3
user_id:17    ,user_idx:4
user_id:18    ,user_idx:5
user_to_idx의 사이즈: 6041


In [24]:
num=0
for key, value in movie_to_idx.items():
    print('movie_id:{}    ,movie_idx:{}'.format(key, value))
    num+=1
    if num>5:
        break
print('user_to_idx의 사이즈:', len(movie_to_idx))

movie_id:1193    ,movie_idx:0
movie_id:661    ,movie_idx:1
movie_id:914    ,movie_idx:2
movie_id:3408    ,movie_idx:3
movie_id:2355    ,movie_idx:4
movie_id:1197    ,movie_idx:5
user_to_idx의 사이즈: 3628


In [25]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = movies_tmp['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(movies_tmp):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    movies_tmp['user_id'] = temp_user_data   # movies_tmp['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

#movie_id를 통해 movie 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = movies_tmp['movie_id'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(movies_tmp):
    print('movie column indexing OK!!')
    movies_tmp['movie_id'] = temp_movie_data
else:
    print('movie column indexing Fail!!')

movies_tmp

user_id column indexing OK!!
movie column indexing OK!!


,user_id,movie_id,count
0,0,0,5
1,1,0,5
2,2,0,4
3,3,0,4
4,4,0,5
...,...,...,...
836483,1621,3623,5
836484,3481,3624,4
836485,3481,3625,3
836486,4159,3626,4


 CSR matrix를 직접 만들어 봅시다.

In [26]:
#SciPy(사이파이)는 과학 컴퓨팅과 기술 컴퓨팅에 사용되는 자유-오픈 소스 파이썬 라이브러리이다
from scipy.sparse import csr_matrix

num_user = movies_tmp['user_id'].nunique()
num_movie = movies_tmp['movie_id'].nunique()


#=> csr_matrix((data, indices, indptr), shape=(row, col))
csr_data = csr_matrix((movies_tmp['count'], (movies_tmp['user_id'], movies_tmp['movie_id'])), shape= (num_user, num_movie))
#csr_data

als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

In [27]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [28]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(\
        factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [29]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6041 sparse matrix of type '<class 'numpy.int64'>'
	with 836488 stored elements in Compressed Sparse Column format>

In [30]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [32]:
# JJEONDA idx값 찾기
JJEONDA_idx_num = user_to_idx['JJEONDA']
print(JJEONDA_idx_num)

#syj의 데이터정보만 가져온다.
is_moives_JJEONDA = movies_tmp['user_id'] == 5677 #56
moives_JJEONDA = movies_tmp[is_moives_JJEONDA]
moives_JJEONDA

5670


,user_id,movie_id,count
40236,5677,40,5
78888,5677,80,5
82320,5677,84,5
106595,5677,110,5
127206,5677,126,5
157763,5677,163,5
196547,5677,204,4
220576,5677,245,5
237380,5677,276,5
253719,5677,306,5


In [34]:
JJEONDA_idx, homealone_idx = user_to_idx['JJEONDA'], movie_to_idx[dir_favorite['Home Alone 2: Lost in New York (1992)']]
JJEONDA_vector, homealone_vector = als_model.user_factors[JJEONDA_idx], als_model.item_factors[homealone_idx]


print('JJEONDA의 인덱스:{}  ,homealone_idx 인덱스:{}'.format(JJEONDA_idx, homealone_idx) )
print('슝=3')

JJEONDA의 인덱스:5670  ,homealone_idx 인덱스:2015
슝=3


In [35]:
JJEONDA_idx

5670

In [36]:
homealone_vector

array([-0.00368237,  0.01100785,  0.01638047, -0.00245515,  0.00814493,
        0.01986632,  0.01423694,  0.01744054,  0.01848664,  0.00830389,
        0.00304481, -0.00521108, -0.01637813,  0.01720892,  0.00216137,
        0.00817107,  0.00207953,  0.01306386,  0.00109178, -0.01025805,
        0.02537838, -0.00378598, -0.00162661, -0.00553135,  0.01249203,
       -0.00093408,  0.00587426,  0.00582796, -0.0011902 , -0.01845865,
        0.00460565, -0.00540353,  0.00591411, -0.00322129,  0.01736781,
       -0.0043687 , -0.00262132,  0.01446025,  0.00013595,  0.01358902,
       -0.00219243,  0.00923886,  0.00795313,  0.00369829,  0.00689605,
       -0.00024051,  0.02065185,  0.01229502,  0.00778025,  0.00751914,
        0.00561945,  0.00558639, -0.0023534 , -0.00723679,  0.00782451,
        0.00276992,  0.0177479 ,  0.00510138,  0.0258256 ,  0.00566899,
       -0.00190447, -0.0019089 , -0.00366359,  0.01114942,  0.00240431,
        0.00045485,  0.01468423, -0.0011598 ,  0.01785812,  0.00

In [37]:
# JJEONDA_idx homealone_vector 내적하는 코드
np.dot(JJEONDA_idx, homealone_vector)

array([ -20.87901582,   62.41452039,   92.87727686,  -13.92070584,
         46.18176897,  112.64202029,   80.72346604,   98.8878553 ,
        104.819267  ,   47.08307745,   17.26408673,  -29.54682337,
        -92.86398031,   97.57460201,   12.25497125,   46.32998482,
         11.79092673,   74.07211332,    6.19036436,  -58.16314182,
        143.89541443,  -21.46649038,   -9.2228546 ,  -31.36277648,
         70.82980969,   -5.29623122,   33.30702966,   33.04450996,
         -6.7484306 , -104.66054276,   26.11404061,  -30.63804331,
         33.53301289,  -18.26471139,   98.47549332,  -24.77055326,
        -14.86286635,   81.98959527,    0.77085574,   77.04976388,
        -12.43110035,   52.38432887,   45.09426129,   20.96928898,
         39.10059397,   -1.36369893,  117.09596263,   69.7127729 ,
         44.11402634,   42.63350744,   31.86230794,   31.6748414 ,
        -13.34377663,  -41.03260144,   44.3649472 ,   15.70542949,
        100.6305797 ,   28.92484537,  146.43116863,   32.14319

내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [38]:
#AlternatingLeastSquares 클래스 
#similar_items 메서드를 통하여 비슷한 영화를 찾습니다
print(my_favorite[1])

favorite_movie = 'Top Gun (1986)'
movie_id = movie_to_idx[title_in_id_out(favorite_movie)]
similar_movie = als_model.similar_items(movie_id, N=10)
similar_movie
#(영화의 idx, 유사도) Tuple 로 반환

Ace Ventura: When Nature Calls (1995)


[(326, 1.0),
 (325, 0.60001904),
 (360, 0.57940257),
 (188, 0.52862614),
 (1219, 0.52597123),
 (614, 0.5131331),
 (1314, 0.49002367),
 (344, 0.4732888),
 (1182, 0.4704993),
 (1243, 0.46560764)]

내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [39]:
#movie_to_idx 를 뒤집어, index로부터 movie_id를  얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}

for i in similar_movie:
    idx = i[0]
    moive_id = idx_to_movie[idx]
    print('idx:{},   moive_id:{} '.format(idx,moive_id))
    print(id_in_title_out(moive_id),'\n')

idx:326,   moive_id:1101 
Top Gun (1986) 

idx:325,   moive_id:2406 
Romancing the Stone (1984) 

idx:360,   moive_id:1088 
Dirty Dancing (1987) 

idx:188,   moive_id:2115 
Indiana Jones and the Temple of Doom (1984) 

idx:1219,   moive_id:2403 
First Blood (1982) 

idx:614,   moive_id:2405 
Jewel of the Nile, The (1985) 

idx:1314,   moive_id:3197 
Presidio, The (1988) 

idx:344,   moive_id:2802 
Tequila Sunrise (1988) 

idx:1182,   moive_id:3441 
Red Dawn (1984) 

idx:1243,   moive_id:2942 
Flashdance (1983) 

